In [1]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## upload on google workbench!!
## upload on github!!!

In [2]:
## Installing Dependencies
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import tensorflow_io as tfio
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from itertools import groupby

print(tf.__version__)
print(tfio.__version__)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

2.12.0
0.31.0


# **STEP 1: Setup DataLoading function:**

In [3]:
def loadAudio(filename): 
    """
    Converts filename into 16k hz mono audio (no stereo audio, easier to process)
    """
    ## Extracting File from Folders
    file = tf.io.read_file(filename)
    
    ## Extracting Audio component, sampling rate for 16k hz conversion
    wave, sampleRate = tf.audio.decode_wav(file, desired_channels=1)
    
    ## Data Processing, removing extra axis in wave data
    wave = tf.squeeze(wave, axis=-1)
    
    ## integerizing sample rate, easier to process later
    sampleRate = tf.cast(sampleRate, dtype=tf.int64)
    
    ## Standardizing wave
    wave = tfio.audio.resample(wave, rate_in=sampleRate, rate_out=16000)
    
    return wave
    
    
    

In [4]:
def load_wav_16k_mono(filename):
    file_contents=tf.io.read_file(filename)
    wav,sample_rate=tf.audio.decode_wav(file_contents,desired_channels=1)
    wav=tf.squeeze(wav,axis=-1)
    sample_rate=tf.cast(sample_rate,dtype=tf.int64)
    wav=tfio.audio.resample(wav,rate_in=sample_rate,rate_out=16000)
    return wav

# Wave Plots

In [5]:

CAPUCHIN_FILE = os.path.join('/kaggle/input','z-by-hp-unlocked-challenge-3-signal-processing','Parsed_Capuchinbird_Clips','XC114131-0.wav')
NOT_CAPUCHIN_FILE= os.path.join('/kaggle/input','z-by-hp-unlocked-challenge-3-signal-processing','Parsed_Not_Capuchinbird_Clips','Crickets-chirping-9.wav')

ieBird = load_wav_16k_mono(CAPUCHIN_FILE)
ieNotBird = load_wav_16k_mono(NOT_CAPUCHIN_FILE)
plt.plot(ieBird)
plt.plot(ieNotBird)
plot.show()

NotImplementedError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/audio_ops.py", line 458, in f
        return core_ops.io_audio_resample(
    File "/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py", line 88, in __getattr__
        return getattr(self._load(), attrb)
    File "/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py", line 84, in _load
        self._mod = _load_library(self._library)
    File "/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py", line 69, in _load_library
        raise NotImplementedError(

    NotImplementedError: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
    caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tensorflow13GcsFileSystemE']


# **Creating TensorFlow Dataset using our DataLoader Function:**

In [ ]:
## Creating Binary Output Data
POS = '/kaggle/input/z-by-hp-unlocked-challenge-3-signal-processing/Parsed_Capuchinbird_Clips'
NEG = '/kaggle/input/z-by-hp-unlocked-challenge-3-signal-processing/Parsed_Not_Capuchinbird_Clips'

In [ ]:
## File paths containing all clips known to hold/not hold bird calls
pos = tf.data.Dataset.list_files(POS+'/*.wav') ## Regular Expression for all files in folder
neg = tf.data.Dataset.list_files(NEG+'/*.wav')


In [ ]:
## Combining previous bernoulli data
positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))
fullData = positives.concatenate(negatives) ## One big tensor of all label data

## We use fulldata as training data for CNN computer vision model

## the 1's or 0's represent the binary output of Yi, where i is the ith image, 
##    Yi is the truth value of the ith image

# **EDA - Calculating Average length of a Capuchin Bird**
- Important, knowing average length => we are capturing the majority of our bird call when slicing through audio files

In [ ]:
## Calculate Wave Cycle Lengths
lengths=[]
for file in os.listdir(os.path.join('data', 'Parsed_Capuchinbird_Clips')):
    
    ## 1d tensor (list) of amplitudes -> length of bird call from training data
    tensorWave=loadAudio(os.path.join('data', 'Parsed_Capuchinbird_Clips', file)) 
    lengths.append(len(tensorWave))

In [ ]:
print(lengths)

In [ ]:
birdCallMean = tf.math.reduce_mean(lengths)

# **PreProcessing**

In [ ]:
def preProcess(filepath, label):
    wav = loadAudio(filepath)
    wav = wav[:48000]
    
    ## Just incase some clips dont meet this length
    zeroPad = tf.zeros([48000 - tf.shape(wav)[0]], dtype=tf.float32)
    wav=tf.concat([zeroPad, wav], 0)
    
    ## Converting to spectogram using Fourier Transform
    spect = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spect = tf.abs(spect)
    
    ## Making sure we are in appropriate datatype
    ## CNN model will expect a channels dimension, current tensor does not have that
    spect = tf.expand_dims(spect, axis=2) 
    return spect, label ## Returning label allows us to use tf.map()

# **Train - Test Datasets**

In [ ]:
## Tensorflow Data Pipeline
data = data.map(preprocess)
data = data.cache() ## What is this?
data = data.shuffle(buffer_size=1000)
data = data.batch(16) ## what is this? ## each index holds 16 samples
data = data.prefetch(8)

In [ ]:
train = data.take(36)
test = data.skip(36).take(15)

# **Building Deep Leaning Model with TensorFlow**

In [ ]:
from tensorflow.keras.models import Sequential ## Simple NN
from tensorflow.keras.layers import Conv2D, Dense, Flatten ## LAYERS for our NN

In [ ]:
model = Sequential()

## input depends on size of spectogram defined earlier
## what is RELU
## EXPLORE THIS FUTHER PLS
## WHY THESE LAYERS? WHY NOT ADD MORE, OR LESS??
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(1491, 257, 1)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile('Adam', loss='BinaryCrossEntropy', 
              metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

model.summary()

# **Fitting, Measuring Loss, KPI Plots**

In [ ]:
histogram = model.fit(train, epochs=4, validation_data=test) ## Explore further also Batch?

In [ ]:
plt.title('Loss')
plt.plot(hist.histor['loss'], 'r')
plt.plot(hist.histor['val_loss'], 'b')
plt.show()

In [ ]:
plt.title('Precision')
plt.plot(hist.histor['prec'], 'r')
plt.plot(hist.histor['val_prec'], 'b')
plt.show()

In [ ]:
plt.title('Recall')
plt.plot(hist.histor['recall'], 'r')
plt.plot(hist.histor['val_recall'], 'b')
plt.show()

# **Forest Clip Parsing Function**

In [ ]:
## Long Audio clips are mp3s
def loadMp3(fileName):
    
    res = tfio.audio.AudioIOTensor(fileName)
    
    tensor = res.to_tensor()
    tensor = tf.math.reduce_sum(tensor, axis=1)/2 ## Single channel now
    
    sampleRate = res.rate
    sampleRate = tf.cast(sampleRate, dtype=tf.int64)
    
    wav=tfio.audio.resample(tensor, rate_in=sampleRate, rate_out=16000)
    return wav

In [ ]:
def preProcessMp3(sample, index):
    sample = sample[0]
    zeroPad = tf.zeros([48000 - tf.shape(sample)[0]], dtype=tf.float32)
    wav = tf.concat([zero_padding, sample], 0)
    specto = tf.signal.stft(wav, frame_length=320, frame_step=32)
    specto = tf.abs(specto)
    specto = tf.expand_dims(specto, axis=2)
    return specto, index

In [ ]:
## Converting mp3 audio into predictable slices
audioSlices = tf.keras.utils.timeseries_dataset_from_array(wav, wav, sequence_length=48000, sequence_stride=48000)
audioSlices = audioSlices.map(preProcessMp3)
audioSlices = audioSlices.batch(64)

In [ ]:
yhat = model.predict(audioSlices)
yhat = [1 if prediction > 0.99 else 0 for prediction in yhat]

# **GROUP BY**
- Multiple consecutive predictions should be grouped into one big prediction

In [ ]:
from itertools import groupby


In [ ]:
yhat = [key for key, group in groupby(yhat)]
calls = tf.math.reduce_sum(yhat).numpy()

# **Final Predictions**

In [ ]:
## Pure predictions

result = {}
for filename in os.listdir(os.path.join('data', 'Forest Recordings')):
    PATH = os.path.join('data', 'Forest Recordings',file)
    
    wav = loadMp3(PATH)
    audioSlices = tf.keras.utils.timeseries_dataset_from_array(wav, wav, sequence_length=48000, sequence_stride=48000)
    audioSlices = audioSlices.map(preProcessMp3)
    audioSlices = audioSlices.batch(64)
    
    yhat = model.predict(audioSlices)
    result[filename] = yhat

In [ ]:
## Converting into 1/0 classes

classes = {}
for file, logits in result.items():
    classes[file] = [1 if prediction > 0.99 else 0 for prediction in yhat]


In [ ]:
## Apply full groupby

final = {}
for file, finalClass in classes.items():
    final[file] = tf.math.reduce_sum([key for key, group in groupby(yhat)]).numpy()

# **Final Export**

In [ ]:
import csv

f = open('results.csv', 'w', newline='')
writer = csv.writer(f, delimiter=',')
writer.writerow(['recording', 'capuchin_calls'])
for key, value in final.items():
    writer.writerow([key, value])
f.close()